Simplified self attention

In [1]:
import torch

In [2]:
inputs = torch.tensor([
    [0.43,0.15,0.89],
    [0.55,0.87,0.66],
    [0.57,0.85,0.64],
    [0.22,0.58,0.33],
    [0.77,0.25,0.10],
    [0.05,0.80,0.55]
])

In [3]:
query = torch.tensor([0.55,0.87,0.66])
inputs.shape[0]

6

In [4]:
attention_scores= torch.zeros(inputs.shape[0])
for i,x_i in enumerate(inputs):
  attention_scores[i]=torch.dot(x_i,query)
attention_scores

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])

why normalization??

sum to 1, Interpretable as probabilities, maintaining training stability in the LLM.


In [5]:
def normal_normalization(x):
  return x/torch.sum(x,dim=-1)
normalized1 = normal_normalization(attention_scores)
print(normalized1)
print(normalized1.sum(dim=-1))

tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
tensor(1.0000)


why soft max??

for 1,2,3,400

i want 400's value to be very clode to 1


1/406 >> e^1/(e^1+e^2+e^3+e^400)



In [6]:
def naive_soft_max(x):
  return torch.exp(x)/torch.exp(x).sum(dim=-1)

normalized2=naive_soft_max(attention_scores)
normalized2

tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])

Why use torch soft max


e^(x-max)/(summation)

it prevents overflow (for large values) and underflow (for small values)

it is optimized too.

In [7]:
normalized3 = torch.softmax(attention_scores,dim=0)
normalized3

tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])

In [8]:
0.1385*torch.tensor([0.43,0.15,0.89])

tensor([0.0596, 0.0208, 0.1233])

In [9]:
context_vector2= torch.zeros(inputs.shape[1])
for i, x_i in enumerate(inputs):
  context_vector2+= normalized3[i]*x_i

In [10]:
context_vector2

tensor([0.4419, 0.6515, 0.5683])

In [11]:
attention_scores=torch.empty(inputs.shape[0],inputs.shape[0])

In [12]:
for i in  range(0,inputs.shape[0]):
  for j in  range(0,inputs.shape[0]):
    attention_scores[i,j] = torch.dot(inputs[i],inputs[j])
attention_scores

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])

In [13]:
efficient_attention_scores = inputs @ inputs.T
efficient_attention_scores

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])

In [14]:
efficient_attention_weights = torch.softmax(efficient_attention_scores,dim=-1)
efficient_attention_weights

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])

In [15]:
efficient_attention_weights.sum(dim=-1)

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])

In [16]:
context_vectors = efficient_attention_weights @ inputs
context_vectors

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])

SELF ATTENTION MECHANISM (Scaled dot product )

In [17]:
inputs = torch.tensor([
    [0.43,0.15,0.89], #your
    [0.55,0.87,0.66], #journey
    [0.57,0.85,0.64], #starts
    [0.22,0.58,0.33], #with
    [0.77,0.25,0.10], #one
    [0.05,0.80,0.55] #step
])

In [18]:
x_2 = inputs[1]
din = x_2.shape[0]
dout = 2

In [19]:
torch.manual_seed(143)
W_query = torch.nn.Parameter(torch.rand(din, dout), requires_grad=False)
W_key = torch.nn.Parameter(torch.rand(din, dout), requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(din, dout), requires_grad=False)

In [20]:
print(W_query)

Parameter containing:
tensor([[0.8653, 0.0099],
        [0.0284, 0.4898],
        [0.8311, 0.6590]])


In [21]:
print(W_key)

Parameter containing:
tensor([[0.5281, 0.7090],
        [0.4898, 0.9040],
        [0.6079, 0.2030]])


In [22]:
print(W_value)

Parameter containing:
tensor([[0.4850, 0.3860],
        [0.2487, 0.9812],
        [0.0049, 0.2948]])


In [23]:
query_2 = x_2 @ W_query
key_2 = x_2 @ W_key
value_2 = x_2 @ W_value
print(query_2)

tensor([1.0491, 0.8665])


In [24]:
print(query_2@key_2.T)

tensor(2.3082)


<ipython-input-24-3aecdaaccea8>:1: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3683.)
  print(query_2@key_2.T)


In [25]:
queries = inputs @ W_query
keys = inputs @ W_key
values = inputs @ W_value

In [26]:
print(queries)
print(keys)
print(values)

tensor([[1.1160, 0.6643],
        [1.0491, 0.8665],
        [1.0492, 0.8437],
        [0.4811, 0.5037],
        [0.7565, 0.1960],
        [0.5231, 0.7548]])
tensor([[0.8416, 0.6212],
        [1.1178, 1.3104],
        [1.1064, 1.3025],
        [0.6009, 0.7473],
        [0.5899, 0.7922],
        [0.7526, 0.8703]])
tensor([[0.2502, 0.5755],
        [0.4863, 1.2605],
        [0.4909, 1.2427],
        [0.2525, 0.7513],
        [0.4361, 0.5720],
        [0.2259, 0.9664]])


In [27]:
attention_scores = queries @ keys.T
attention_scores

tensor([[1.3518, 2.1179, 2.0999, 1.1670, 1.1846, 1.4180],
        [1.4212, 2.3082, 2.2894, 1.2780, 1.3054, 1.5437],
        [1.4071, 2.2785, 2.2599, 1.2610, 1.2874, 1.5240],
        [0.7178, 1.1979, 1.1884, 0.6655, 0.6829, 0.8005],
        [0.7584, 1.1024, 1.0922, 0.6010, 0.6015, 0.7399],
        [0.9091, 1.5738, 1.5618, 0.8784, 0.9065, 1.0506]])

In [28]:
scaled_attention_scores = attention_scores/(keys.shape[1])**0.5
scaled_attention_scores

tensor([[0.9559, 1.4976, 1.4849, 0.8252, 0.8376, 1.0027],
        [1.0049, 1.6322, 1.6188, 0.9036, 0.9230, 1.0916],
        [0.9950, 1.6112, 1.5980, 0.8917, 0.9103, 1.0776],
        [0.5076, 0.8470, 0.8403, 0.4706, 0.4829, 0.5660],
        [0.5363, 0.7795, 0.7723, 0.4250, 0.4253, 0.5232],
        [0.6428, 1.1129, 1.1044, 0.6211, 0.6410, 0.7429]])

In [29]:
attention_weights = torch.softmax(scaled_attention_scores,dim=-1)
attention_weights

tensor([[0.1383, 0.2378, 0.2347, 0.1214, 0.1229, 0.1449],
        [0.1310, 0.2452, 0.2420, 0.1183, 0.1207, 0.1428],
        [0.1318, 0.2441, 0.2409, 0.1189, 0.1211, 0.1432],
        [0.1471, 0.2065, 0.2052, 0.1418, 0.1435, 0.1559],
        [0.1583, 0.2019, 0.2004, 0.1416, 0.1417, 0.1562],
        [0.1376, 0.2201, 0.2183, 0.1346, 0.1373, 0.1521]])

In [30]:
context_vectors= attention_weights @ values
context_vectors

tensor([[0.3824, 0.9726],
        [0.3856, 0.9811],
        [0.3851, 0.9799],
        [0.3716, 0.9393],
        [0.3690, 0.9330],
        [0.3769, 0.9546]])

In [31]:
import torch.nn as nn

class selfAttention_V1(nn.Module):

  def __init__(self, din, dout):
        super().__init__()
        self.W_query = torch.nn.Parameter(torch.rand(din, dout))
        self.W_key = torch.nn.Parameter(torch.rand(din, dout))
        self.W_value = torch.nn.Parameter(torch.rand(din, dout))

  def forward(self, x):
        queries = x @ self.W_query
        keys = x @ self.W_key
        values = x @ self.W_value

        attention_socres = queries @ keys.T
        attention_weights = torch.softmax((attention_socres)/(keys.shape[1])**0.5,dim=-1)

        context_vec = attention_weights @ values

        return context_vec


In [32]:
torch.manual_seed(143)
sa_v1 = selfAttention_V1(din,dout)
print(sa_v1.forward(inputs))

tensor([[0.3824, 0.9726],
        [0.3856, 0.9811],
        [0.3851, 0.9799],
        [0.3716, 0.9393],
        [0.3690, 0.9330],
        [0.3769, 0.9546]], grad_fn=<MmBackward0>)


In [33]:
class selfAttention_V2(nn.Module):

  def __init__(self, din, dout, qkv_bias=False):
        super().__init__()
        self.W_query = nn.Linear(din,dout, bias=qkv_bias )
        self.W_key = nn.Linear(din, dout, bias= qkv_bias)
        self.W_value = nn.Linear(din, dout, bias=qkv_bias)

  def forward(self, x):
        queries = self.W_query(x)
        keys = self.W_key(x)
        values = self.W_value(x)

        attention_socres = queries @ keys.T
        attention_weights = torch.softmax((attention_socres)/(keys.shape[1])**0.5,dim=-1)

        context_vec = attention_weights @ values

        return context_vec


In [34]:
torch.manual_seed(789)
sa_v2 = selfAttention_V2(din,dout)
print(sa_v2.forward(inputs))

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)


Causual Attention

Masked self attention

In [35]:
din = inputs.shape[1]
dout = 2
sa_v2 = selfAttention_V2(din,dout)
queries = sa_v2.W_query(inputs)
keys =  sa_v2.W_key(inputs)
values = sa_v2.W_value(inputs)
attention_scores=queries @ keys.T
attention_weights = torch.softmax(attention_scores/(keys.shape[1])**0.5,dim=-1)

In [36]:
context_length = attention_weights.shape[1]
mask = torch.tril(torch.ones(context_length,context_length))
masked_attention_weights = attention_weights*mask
masked_attention_weights

tensor([[0.1766, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1772, 0.1720, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1769, 0.1719, 0.1716, 0.0000, 0.0000, 0.0000],
        [0.1725, 0.1696, 0.1695, 0.1618, 0.0000, 0.0000],
        [0.1687, 0.1694, 0.1692, 0.1637, 0.1634, 0.0000],
        [0.1758, 0.1704, 0.1702, 0.1598, 0.1615, 0.1623]],
       grad_fn=<MulBackward0>)

In [37]:
rows_sum = masked_attention_weights.sum(dim=1, keepdim=True)
normalized_masked_attention_weights= masked_attention_weights/rows_sum
normalized_masked_attention_weights

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5075, 0.4925, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3399, 0.3303, 0.3298, 0.0000, 0.0000, 0.0000],
        [0.2562, 0.2519, 0.2517, 0.2402, 0.0000, 0.0000],
        [0.2021, 0.2030, 0.2028, 0.1962, 0.1959, 0.0000],
        [0.1758, 0.1704, 0.1702, 0.1598, 0.1615, 0.1623]],
       grad_fn=<DivBackward0>)

In [38]:
din = inputs.shape[1]
dout = 2
sa_v2 = selfAttention_V2(din,dout)
queries = sa_v2.W_query(inputs)
keys =  sa_v2.W_key(inputs)
values = sa_v2.W_value(inputs)
attention_scores=queries @ keys.T

In [39]:
mask = torch.triu(torch.ones(context_length,context_length),diagonal=1)
masked = attention_scores.masked_fill(mask.bool(), -torch.inf)
masked

tensor([[-0.0559,    -inf,    -inf,    -inf,    -inf,    -inf],
        [-0.1436, -0.1670,    -inf,    -inf,    -inf,    -inf],
        [-0.1324, -0.1541, -0.1474,    -inf,    -inf,    -inf],
        [-0.1134, -0.1314, -0.1263, -0.0793,    -inf,    -inf],
        [ 0.1071,  0.1220,  0.1191,  0.0679,  0.0330,    -inf],
        [-0.2257, -0.2607, -0.2512, -0.1550, -0.0094, -0.2561]],
       grad_fn=<MaskedFillBackward0>)

In [40]:
attention_weights = torch.softmax(masked/(keys.shape[1])**0.5,dim=-1)
attention_weights

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5041, 0.4959, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3362, 0.3311, 0.3327, 0.0000, 0.0000, 0.0000],
        [0.2498, 0.2467, 0.2476, 0.2559, 0.0000, 0.0000],
        [0.2024, 0.2045, 0.2041, 0.1969, 0.1921, 0.0000],
        [0.1625, 0.1585, 0.1596, 0.1709, 0.1894, 0.1591]],
       grad_fn=<SoftmaxBackward0>)

In [41]:
batch = torch.stack((inputs,inputs),dim=0)
batch

tensor([[[0.4300, 0.1500, 0.8900],
         [0.5500, 0.8700, 0.6600],
         [0.5700, 0.8500, 0.6400],
         [0.2200, 0.5800, 0.3300],
         [0.7700, 0.2500, 0.1000],
         [0.0500, 0.8000, 0.5500]],

        [[0.4300, 0.1500, 0.8900],
         [0.5500, 0.8700, 0.6600],
         [0.5700, 0.8500, 0.6400],
         [0.2200, 0.5800, 0.3300],
         [0.7700, 0.2500, 0.1000],
         [0.0500, 0.8000, 0.5500]]])

In [42]:
batch.shape

torch.Size([2, 6, 3])

In [43]:
class causualAttention_V1(nn.Module):

  def __init__(self, din, dout, context_length, dropout, qkv_bias=False):
        super().__init__()
        self.W_query = nn.Linear(din,dout, bias=qkv_bias )
        self.W_key = nn.Linear(din, dout, bias= qkv_bias)
        self.W_value = nn.Linear(din, dout, bias=qkv_bias)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer('mask',torch.triu(torch.ones(context_length,context_length),diagonal=1))

  def forward(self, x):
        batch, num_tokens, din= x.shape
        queries = self.W_query(x)
        keys = self.W_key(x)
        values = self.W_value(x)

        attention_socres = queries @ keys.transpose(1,2)

        attention_scores.masked_fill(self.mask.bool()[:num_tokens,:num_tokens], -torch.inf)

        attention_weights = torch.softmax((attention_scores)/(keys.shape[2])**0.5,dim=-1)

        attention_weights = self.dropout(attention_weights)

        context_vec = attention_weights @ values

        return context_vec


In [44]:
torch.manual_seed(123)
context_length = batch.shape[1]
ca = causualAttention_V1(din,dout,context_length,0.5)
context_vec_matrix = ca.forward(batch)
context_vec_matrix

tensor([[[-0.5006, -0.1663],
         [-0.5575, -0.0497],
         [-0.5566, -0.0488],
         [-0.7670, -0.2376],
         [-0.1413, -0.0503],
         [-0.6915, -0.0984]],

        [[-0.5006, -0.1663],
         [-0.5575, -0.0497],
         [-0.5566, -0.0488],
         [-0.7670, -0.2376],
         [-0.1413, -0.0503],
         [-0.6915, -0.0984]]], grad_fn=<CloneBackward0>)

Multi head attention

In [61]:
class multiheadattention(nn.Module):

  def __init__(self,din, dout, context_length, dropout, num_heads, qkv_bias=False):

    super().__init__()
    assert(dout%num_heads==0),\
        "dout must be divisible by num_heads"
    self.W_query = nn.Linear(din, dout, bias=qkv_bias)
    self.W_key = nn.Linear(din, dout, bias=qkv_bias)
    self.W_value = nn.Linear(din, dout, bias=qkv_bias)
    self.dropout = nn.Dropout(dropout)
    self.num_heads = num_heads
    self.head_dim = dout//num_heads
    self.register_buffer('mask',torch.triu(torch.ones(context_length,context_length),diagonal=1))

  def forward(self,x):
      b, num_tokens, dout = x.shape

      queries = self.W_query(x)
      keys = self.W_key(x)
      values = self.W_value(x)

      queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)
      keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
      values = values.view(b, num_tokens, self.num_heads, self.head_dim)

      queries = queries.transpose(1,2)
      keys = keys.transpose(1,2)
      values = values.transpose(1,2)

      attention_scores = queries @ keys.transpose(2,3)
      attention_scores.masked_fill(self.mask.bool()[:num_tokens, :num_tokens],-torch.inf)
      attention_weights = torch.softmax(attention_scores/(keys.shape[-1])**0.5, dim=-1)
      attention_weights = self.dropout(attention_weights)
      context_vectors = (attention_weights @ values).transpose(1,2)
      context_vectors = context_vectors.contiguous().view(b,num_tokens,dout)
      return context_vectors


In [66]:
torch.manual_seed(123)
inputs = torch.tensor([[0.43,0.15,0.89, 0.55,0.87,0.66],
    [0.57,0.85,0.64, 0.22,0.58,0.33],
    [0.77,0.25,0.10, 0.05,0.80,0.55]])

batch = torch.stack((inputs,inputs))
dout = 6
b, context_length, din = batch.shape
mha = multiheadattention(din, dout, context_length, 0.0, num_heads=2)
context_vectors=mha.forward(batch)
print(context_vectors)
print(context_vectors.shape)

tensor([[[-0.2899,  0.2503,  0.0068,  0.0055,  0.5636,  0.0698],
         [-0.2886,  0.2483,  0.0094,  0.0062,  0.5648,  0.0684],
         [-0.2898,  0.2501,  0.0070,  0.0092,  0.5663,  0.0665]],

        [[-0.2899,  0.2503,  0.0068,  0.0055,  0.5636,  0.0698],
         [-0.2886,  0.2483,  0.0094,  0.0062,  0.5648,  0.0684],
         [-0.2898,  0.2501,  0.0070,  0.0092,  0.5663,  0.0665]]],
       grad_fn=<ViewBackward0>)
torch.Size([2, 3, 6])
